# S1 TDS analysis

In [1]:
import pandas as pd
import scipy.constants
from scipy.optimize import curve_fit
from scipy.integrate import quad
from sklearn.metrics import r2_score
import seaborn
seaborn.set(style='whitegrid')
import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib.dates as md
import matplotlib.transforms as transforms
from matplotlib.ticker import LogFormatter
%matplotlib qt
# %matplotlib inline
import datetime as dt
pd.set_option('display.max_rows', 100) 

In [2]:
#reading vaclogger measurement file
vaclog=pd.read_csv("vaclog", sep="\t")


#adding an elapsed time column
vac_timestamps=pd.to_datetime(vaclog["Time"],format="%d/%m/%Y %H:%M:%S")
runtime=(vac_timestamps-vac_timestamps[0]).dt.total_seconds()    
vaclog.insert(2,"Elapsed time",runtime)
# vaclog.drop(index=vaclog.index[0], 
#         axis=0, 
#         inplace=True)
vaclog["Time"] = vac_timestamps.dt.strftime('%d-%m-%Y %H:%M:%S')
vaclog["Time"] = pd.to_datetime(vaclog["Time"],format='%d-%m-%Y %H:%M:%S')

vaclog.head()

,Live comments,Time,Elapsed time,100mbar inj,Barion_2,Barion_1,DUAL experiment,DUAL insulation,1mbar inj,helium,T-platinum,T-CERNOX,V_heater,DUAL inj line,I_emission,I_grid
0,NaN,2023-04-28 16:54:36,0.0,-0.053862,3.780000e-09,1.540000e-09,4.996000e-09,5.198000e-07,0.006776,299.0,-1.796,4269.568,0.000031,3.220000e-07,NaN,NaN
1,NaN,2023-04-28 16:54:45,9.0,-0.053763,3.770000e-09,1.540000e-09,4.996000e-09,5.198000e-07,0.006795,299.0,-2.020,4269.455,0.000034,3.220000e-07,NaN,NaN
2,NaN,2023-04-28 16:54:54,18.0,-0.053272,3.770000e-09,1.530000e-09,4.996000e-09,5.198000e-07,0.006795,299.0,-1.908,4269.792,0.000031,3.220000e-07,NaN,NaN
3,NaN,2023-04-28 16:55:02,26.0,-0.053123,3.760000e-09,1.540000e-09,4.996000e-09,5.204000e-07,0.006818,300.0,-1.908,4269.568,0.000036,3.230000e-07,NaN,NaN
4,NaN,2023-04-28 16:55:11,35.0,-0.053689,3.760000e-09,1.540000e-09,4.996000e-09,5.198000e-07,0.006779,299.0,-2.020,4269.904,0.000039,3.230000e-07,NaN,NaN


In [3]:
#Temperature curve for CERNOX - for temp stability
A=[230.317302,-6170.1513,71837.9529,-477946.76,2.003668910085786e+6,-5.488690193047771e+6,9.830475663897528e+6,-1.111226817786569e+7,7.202477878914065e+6,-2.04194551328507e+6]
#specify fit parameters A, data (Resistance values)
def polyfit(params,data):
    total=[]
    for j in data: 
        exp=0
        for i in range(len(params)):
            exp += (params[i]/(math.log10(j))**i)
        total.append(10**exp)
    return(total)    


In [4]:
#print hv comments
print(pd.unique(vaclog["Live comments"]))
vaclog_comm = vaclog.loc[:,["Time","Live comments"]]
vaclog_comm.dropna(inplace=True)
print(vaclog_comm)
vaclog_comm.index

[nan 'closing baryons' 'closed turbo valve']
                  Time       Live comments
29 2023-04-28 16:58:46     closing baryons
47 2023-04-28 17:01:21  closed turbo valve


Int64Index([29, 47], dtype='int64')

### Plotting

In [5]:
#define parameters for plotting
#annotation params
font = dict(size = "x-small", color ="green", style ="italic",rotation="vertical")


In [6]:
#Plotting BA2, BA1 pressure
plt.figure(figsize=(12,6))
plt.plot(vaclog["Time"],vaclog["Barion_2"],marker=".", markersize=4,label='Barion 2')
plt.plot(vaclog["Time"],vaclog["Barion_1"],marker=".", markersize=4,label='Barion 1')
plt.plot(vaclog["Time"],vaclog["DUAL experiment"],marker=".", markersize=4,label='Dual exp')
plt.plot(vaclog["Time"],vaclog["helium"],marker=".", markersize=4,label='Dual exp')

plt.xlabel('Timestamp')
plt.ylabel('Pressure (mbar)')
plt.legend()
plt.title("Total pressure evolution during bakeout")
plt.tick_params(axis="y", which='minor')
plt.grid(which='minor', axis='y')
plt.yscale('log')

ax = plt.gca()
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(10))
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')  

#comments
trans  = transforms.blended_transform_factory(
    ax.transData, ax.transAxes)
for i in vaclog_comm.index:
    plt.text(vaclog_comm["Time"][i],0.8,str(vaclog_comm["Live comments"][i]),fontdict=font, transform=trans)
    
plt.savefig(r'./graphs/TDS pressures.png')
plt.show()

In [ ]:
## Plotting the temp evolution
plt.figure(figsize=(10,6))
plt.plot((vaclog["Time"]),polyfit(A,vaclog["T-CERNOX"]),marker=".", markersize=5,label='T-CERNOX')          
plt.xlabel('Timestamp')
plt.ylabel('Temperature (K)')
plt.legend()
plt.title("Temperature evolution")
plt.yscale('linear')
plt.tick_params(axis="y", which='minor')
plt.grid(which='minor', axis='y')

ax = plt.gca()
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(10))
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')
    
plt.savefig(r'./graphs/Temp.png')
plt.show()